In [88]:
import math
import numpy as np
import random as rand
from numba import jit,int64,float64
import matplotlib.pyplot as plt
import scipy.optimize as sciopt
%matplotlib inline

#To increase cell width:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#Color-blind friendly colors to plot:
# CB_colors = ['#377eb8', '#ff7f00', '#4daf4a','#f781bf', '#a65628', '#984ea3','#999999', '#e41a1c', '#dede00']
# CB_colors.reverse()
CB_colors = ['#00429d', '#93003a']

In [2]:
#The spacetime parameters. Let the population distribution of gene expression level be g:
g_min = 0
g_max = 100
dg = 0.4
#Defining a g_ghost which contains two extra `ghost' points at the edges:
g_ghost = np.arange(g_min-dg,g_max+2*dg,dg)
g = np.array(g_ghost[1:-1])
#The peak of the unregulated distribution:
g_peak = 0.5*(g_min+g_max)

#Parameters:
K = 1
delta = 100
alpha = 0.5

#Constant environment, T fixed:
dt = np.minimum(0.00005/K,0.01/delta)
T = 10
# #If tau is too large, we don't change:
# num_cycles = 8
# T = num_cycles*tau
TimeRange = np.arange(0,T,dt)
print(len(TimeRange))

#Tolerance to calculate entropy:
eps = 1e-100

#The noise variance D:
D = alpha*K*g_peak

#Checks:
print(f"K = {K}; 0.5*dg/dt = {0.5*dg/dt}")
print(f"D = {D}; 0.5*dg**2/dt = {0.5*dg**2/dt}")

#Stability check:
flag_stability=0
if (K>=int(0.5*dg/dt) or D>=int(0.5*dg**2/dt)):
    flag_stability=1
    print("Warning! FTCS unstable.")

200000
K = 1; 0.5*dg/dt = 4000.0
D = 25.0; 0.5*dg**2/dt = 1600.0000000000002


In [99]:
#First derivative, central difference method (used in the potential term K*d/dg (g*P)):
@jit("float64[:](float64[:],float64)",nopython=True)
def derv1(func,dx):
    func_left = func[0:-2]
    func_right = func[2:]
    #Below we calculate the first derivative using the central difference method:
    derivative = (func_right - func_left)/(2*dx)
    return derivative

#Second derivative using central difference, used in the diffusion term (D*d2/dg2 (P)):
@jit("float64[:](float64[:],float64)",nopython=True)
def derv2(func,dx):
    func_left = func[0:-2]
    func_right = func[2:]
    func_center = func[1:-1]
    #Below we calculate the second derivative, again using central difference method:
    derivative2 = (func_right + func_left - 2*func_center)/(dx**2)
    return derivative2

#Defining a Gaussian pdf - we'll need it for P(g,t=0), the initial distribution:
@jit(nopython=True)
def Gaussian(x,mu,sigma):
    dist = 1/(sigma*np.sqrt(2*np.pi))*np.exp(-0.5*((x-mu)/sigma)**2)
    return dist

#The function to calculate <f> dynamically, i.e. int_g f(g,s) P(g,t):
@jit(nopython=True)
def mean_wrt_P(func,P,g):
    integrand = func*P
    integral = np.trapz(integrand,g)
    return integral

#Defining the fitness function:
# @jit(["float64[:](float64[:],float64,float64)","float64(float64,float64,float64)"],nopython=True)
def fitness(g,s,delta):
    fit = delta*(g*s/(400+s) - (2/17)*g/(1-g/(1.8*g_max)))/g_max
    return fit

@jit(nopython=True)
def discrete_entropy(p_dist):
    p_dist = p_dist[np.where(np.clip(p_dist,eps,None)>eps)]
    return np.sum(p_dist*np.log2(1/p_dist))

@jit(nopython=True)
def cont_entropy(p_dist):
    p_dist = p_dist[np.where(np.clip(p_dist,eps,None)>eps)]
    return np.trapz(p_dist*np.log2(1/p_dist),dx=dg)

@jit(nopython=True)
def selection_const_noise(p0_unreg,p0_reg,s,K,alpha,delta,T,dt):
    #First, time:
#     dt = 0.0001/K
    TimeRange = np.arange(0,T,dt)
    
    #Then, define the fitness function:
    fit = delta*(g*s/(400+s) - (2/17)*g/(1-g/(1.8*g_max)))/g_max
    g_reg = g[np.argmax(fit)]

    #Now let's compare the effects of regulation vs selection:
    p_unreg = p0_unreg
    p_reg = p0_reg
    flag_unreg = 0
    flag_reg = 0
    
#     #Storing dynamic fitness:
#     dyn_fit_unreg = np.zeros_like

    #Now evolve it:
    for t in range(len(TimeRange)):
        #Constant noise:
        D_unreg = alpha*K*g_peak
        D_reg = alpha*K*g_peak

    #     #Checks:
    #     if (t%10==0):
    #         print(f"\nt={t}; g_mean_unreg={round(g_mean_unreg,2)}; D_unreg={round(D_unreg,2)}; max allowed={0.5*dg**2/dt}")
    #         print(f"t={t}; g_mean_reg={round(g_mean_reg,2)}; D_reg={round(D_reg,2)}; max allowed={0.5*dg**2/dt}")        

        #Now creating the expanded p's, with ghost points. The bulk points are the same in each. Unregulated:
        p_unreg_ghost = np.zeros(len(p_unreg)+2)
        p_unreg_ghost[1:-1] = p_unreg
        #Regulated:
        p_reg_ghost = np.zeros(len(p_reg)+2)
        p_reg_ghost[1:-1] = p_reg

        #Setting the value of the ghost points. This represents the zero flux boundary conditions. Unregulated:
        p_unreg_ghost[0] = p_unreg[1] + 2*dg*(K/D_unreg)*(g[0]-g_peak)*p_unreg[0]
        p_unreg_ghost[-1] = p_unreg[-2] + 2*dg*(K/D_unreg)*(g_peak-g[-1])*p_unreg[-1]
        #Regulated:
        p_reg_ghost[0] = p_reg[1] + 2*dg*(K/D_reg)*(g[0]-g_reg)*p_reg[0]
        p_reg_ghost[-1] = p_reg[-2] + 2*dg*(K/D_reg)*(g_reg-g[-1])*p_reg[-1]    

        #Now to solve the equation. Unregulated first:
        p_unreg += dt*((fit - mean_wrt_P(fit,p_unreg,g))*p_unreg \
                    + K*derv1((g_ghost-g_peak)*p_unreg_ghost,dg) + D_unreg*derv2(p_unreg_ghost,dg))
        p_reg += dt*((fit - mean_wrt_P(fit,p_reg,g))*p_reg \
                    + K*derv1((g_ghost-g_reg)*p_reg_ghost,dg) + D_reg*derv2(p_reg_ghost,dg))

#         #Finally, for the edge cases, we normalize:
#         if (flag_unreg==1 or delta>=1):
#             p_unreg = p_unreg/np.trapz(p_unreg,dx=dg)
#         if (flag_reg==1 or delta>=1):
#             p_reg = p_reg/np.trapz(p_reg,dx=dg)
            
    return p_unreg,p_reg

@jit(nopython=True)
def selection_dyn_const_noise(p0_unreg,p0_reg,sug_dyn,K,alpha,delta,T,dt):
    #First, time:
    TimeRange = np.arange(0,T,dt)    
    
    #Then, define the fitness function:
    s = sug_dyn[0]
    fit = delta*(g*s/(400+s) - (2/17)*g/(1-g/(1.8*g_max)))/g_max
    g_reg = g[np.argmax(fit)]

    #Now let's compare the effects of regulation vs selection:
    p_unreg = p0_unreg
    p_reg = p0_reg
    flag_unreg = 0
    flag_reg = 0
    
    #Storing dynamic fitness:
    dyn_fit_unreg, dyn_norm_fit_unreg, dyn_fit_reg, dyn_norm_fit_reg = np.zeros_like(TimeRange), np.zeros_like(TimeRange), np.zeros_like(TimeRange), np.zeros_like(TimeRange)
    #Dynamic distributions:
    dyn_p_unreg, dyn_p_reg = np.zeros((len(TimeRange),len(g))), np.zeros((len(TimeRange),len(g)))
    dyn_p_unreg[0,:], dyn_p_reg[0,:] = p_unreg, p_reg    

    #Now evolve it:
    for t in range(len(TimeRange)):
        #First setting the dynamic sugar and g_reg:
        s = sug_dyn[t]
        fit = delta*(g*s/(400+s) - (2/17)*g/(1-g/(1.8*g_max)))/g_max
        g_reg = g[np.argmax(fit)]
            
        #Constant noise:
        D_unreg = alpha*K*g_peak
        D_reg = alpha*K*g_peak

    #     #Checks:
    #     if (t%10==0):
    #         print(f"\nt={t}; g_mean_unreg={round(g_mean_unreg,2)}; D_unreg={round(D_unreg,2)}; max allowed={0.5*dg**2/dt}")
    #         print(f"t={t}; g_mean_reg={round(g_mean_reg,2)}; D_reg={round(D_reg,2)}; max allowed={0.5*dg**2/dt}")        

        #Now creating the expanded p's, with ghost points. The bulk points are the same in each. Unregulated:
        p_unreg_ghost = np.zeros(len(p_unreg)+2)
        p_unreg_ghost[1:-1] = p_unreg
        #Regulated:
        p_reg_ghost = np.zeros(len(p_reg)+2)
        p_reg_ghost[1:-1] = p_reg

        #Setting the value of the ghost points. This represents the zero flux boundary conditions. Unregulated:
        p_unreg_ghost[0] = p_unreg[1] + 2*dg*(K/D_unreg)*(g[0]-g_peak)*p_unreg[0]
        p_unreg_ghost[-1] = p_unreg[-2] + 2*dg*(K/D_unreg)*(g_peak-g[-1])*p_unreg[-1]
        #Regulated:
        p_reg_ghost[0] = p_reg[1] + 2*dg*(K/D_reg)*(g[0]-g_reg)*p_reg[0]
        p_reg_ghost[-1] = p_reg[-2] + 2*dg*(K/D_reg)*(g_reg-g[-1])*p_reg[-1]    

        #Now to solve the equation. Unregulated first:
        p_unreg += dt*((fit - mean_wrt_P(fit,p_unreg,g))*p_unreg \
                    + K*derv1((g_ghost-g_peak)*p_unreg_ghost,dg) + D_unreg*derv2(p_unreg_ghost,dg))
        p_reg += dt*((fit - mean_wrt_P(fit,p_reg,g))*p_reg \
                    + K*derv1((g_ghost-g_reg)*p_reg_ghost,dg) + D_reg*derv2(p_reg_ghost,dg))

#         #Finally, for the edge cases, we normalize:
#         if (flag_unreg==1 or delta>=1): #Because apparently this blows up temporarily when delta>1
#             p_unreg = p_unreg/np.trapz(p_unreg,dx=dg)
#         if (flag_reg==1 or delta>=1):
#             p_reg = p_reg/np.trapz(p_reg,dx=dg)
            
        #Storing the dynamic distributions:
        dyn_p_unreg[t,:], dyn_p_reg[t,:] = p_unreg, p_reg
            
        #Now calculating the dynamic fitnesses, absolute:
        dyn_fit_unreg[t] = mean_wrt_P(fit,p_unreg,g)
        dyn_fit_reg[t] = mean_wrt_P(fit,p_reg,g)
        #Normalized:
        dyn_norm_fit_unreg[t] = (dyn_fit_unreg[t] - np.min(fit))/(np.max(fit) - np.min(fit))
        dyn_norm_fit_reg[t] = (dyn_fit_reg[t] - np.min(fit))/(np.max(fit) - np.min(fit))
            
    return dyn_p_unreg, dyn_p_reg, dyn_fit_unreg, dyn_fit_reg

@jit(nopython=True)
def selection_dyn_scalar_output(p0_unreg,p0_reg,sug_dyn,K,alpha,delta,T,dt):
    #First, time:
    TimeRange = np.arange(0,T,dt)    
    
    #Then, define the fitness function:
    s = sug_dyn[0]
    fit = delta*(g*s/(400+s) - (2/17)*g/(1-g/(1.8*g_max)))/g_max
    g_reg = g[np.argmax(fit)]

    #Now let's compare the effects of regulation vs selection:
    p_unreg = p0_unreg
    p_reg = p0_reg
    flag_unreg = 0
    flag_reg = 0
    
    #Storing dynamic fitness:
    dyn_fit_unreg, dyn_norm_fit_unreg, dyn_fit_reg, dyn_norm_fit_reg = np.zeros_like(TimeRange), np.zeros_like(TimeRange), np.zeros_like(TimeRange), np.zeros_like(TimeRange)
    #Dynamic distributions:
    dyn_p_unreg, dyn_p_reg = np.zeros((len(TimeRange),len(g))), np.zeros((len(TimeRange),len(g)))
    dyn_p_unreg[0,:], dyn_p_reg[0,:] = p_unreg, p_reg    

    #Now evolve it:
    for t in range(len(TimeRange)):
        #First setting the dynamic sugar and g_reg:
        s = sug_dyn[t]
        fit = delta*(g*s/(400+s) - (2/17)*g/(1-g/(1.8*g_max)))/g_max
        g_reg = g[np.argmax(fit)]
            
        #Constant noise:
        D_unreg = alpha*K*g_peak
        D_reg = alpha*K*g_peak

    #     #Checks:
    #     if (t%10==0):
    #         print(f"\nt={t}; g_mean_unreg={round(g_mean_unreg,2)}; D_unreg={round(D_unreg,2)}; max allowed={0.5*dg**2/dt}")
    #         print(f"t={t}; g_mean_reg={round(g_mean_reg,2)}; D_reg={round(D_reg,2)}; max allowed={0.5*dg**2/dt}")        

        #Now creating the expanded p's, with ghost points. The bulk points are the same in each. Unregulated:
        p_unreg_ghost = np.zeros(len(p_unreg)+2)
        p_unreg_ghost[1:-1] = p_unreg
        #Regulated:
        p_reg_ghost = np.zeros(len(p_reg)+2)
        p_reg_ghost[1:-1] = p_reg

        #Setting the value of the ghost points. This represents the zero flux boundary conditions. Unregulated:
        p_unreg_ghost[0] = p_unreg[1] + 2*dg*(K/D_unreg)*(g[0]-g_peak)*p_unreg[0]
        p_unreg_ghost[-1] = p_unreg[-2] + 2*dg*(K/D_unreg)*(g_peak-g[-1])*p_unreg[-1]
        #Regulated:
        p_reg_ghost[0] = p_reg[1] + 2*dg*(K/D_reg)*(g[0]-g_reg)*p_reg[0]
        p_reg_ghost[-1] = p_reg[-2] + 2*dg*(K/D_reg)*(g_reg-g[-1])*p_reg[-1]    

        #Now to solve the equation. Unregulated first:
        p_unreg += dt*((fit - mean_wrt_P(fit,p_unreg,g))*p_unreg \
                    + K*derv1((g_ghost-g_peak)*p_unreg_ghost,dg) + D_unreg*derv2(p_unreg_ghost,dg))
        p_reg += dt*((fit - mean_wrt_P(fit,p_reg,g))*p_reg \
                    + K*derv1((g_ghost-g_reg)*p_reg_ghost,dg) + D_reg*derv2(p_reg_ghost,dg))

#         #Finally, for the edge cases, we normalize:
#         if (flag_unreg==1 or delta>=1): #Because apparently this blows up temporarily when delta>1
#             p_unreg = p_unreg/np.trapz(p_unreg,dx=dg)
#         if (flag_reg==1 or delta>=1):
#             p_reg = p_reg/np.trapz(p_reg,dx=dg)
            
        #Storing the dynamic distributions:
        dyn_p_unreg[t,:], dyn_p_reg[t,:] = p_unreg, p_reg
            
        #Now calculating the dynamic fitnesses, absolute:
        dyn_fit_unreg[t] = mean_wrt_P(fit,p_unreg,g)
        dyn_fit_reg[t] = mean_wrt_P(fit,p_reg,g)
        #Normalized:
        dyn_norm_fit_unreg[t] = (dyn_fit_unreg[t] - np.min(fit))/(np.max(fit) - np.min(fit))
        dyn_norm_fit_reg[t] = (dyn_fit_reg[t] - np.min(fit))/(np.max(fit) - np.min(fit))
        #The long-term fitnesses:
        long_term_fit_unreg = np.trapz(dyn_fit_unreg,dx=dt)
        long_term_fit_reg = np.trapz(dyn_fit_reg,dx=dt)
            
    return long_term_fit_unreg, long_term_fit_reg

def logistic_sigmoid(x,height,pos,slope):
    return height/(1 + np.exp(-slope*(x-pos)))

def exponential_growth(x,a,b):
    return a*np.exp(b*x)

def linear_growth(x,a,b):
    return a*x + b

In [4]:
# #Now we initialize the parameters:
# K = 0.01
# delta = 1
# alpha = 0.5
# tau = 100

# #Constant environment, T fixed:
# dt = np.minimum(0.00005/K,0.005/delta)
# # dt = 0.0001/K
# # T = 10
# # #If tau is too large, we don't change:
# num_cycles = 16
# T = num_cycles*tau
# TimeRange = np.arange(0,T,dt)
# print("No. of timepoints: ",len(TimeRange))
# print("dt: ",dt)

# # The noise variance D:
# D = alpha*K*g_peak

# #Checks:
# print(f"K = {K}; 0.5*dg/dt = {0.5*dg/dt}")
# print(f"D = {D}; 0.5*dg**2/dt = {0.5*dg**2/dt}")

# #Stability check:
# flag_stability=0
# if (K>=int(0.5*dg/dt) or D>=int(0.5*dg**2/dt)):
#     flag_stability=1
#     print("Warning! FTCS unstable.")

# #The initial distributions:
# p0_unreg = Gaussian(g,g_peak,np.maximum(np.sqrt(alpha*g_peak),1))
# p0_unreg = p0_unreg/np.trapz(p0_unreg,dx=dg)
# p0_reg = p0_unreg
# p0_reg = p0_reg/np.trapz(p0_reg,dx=dg)

In [5]:
# flag = False
# sug_dyn = np.ones_like(TimeRange)
# temp_sug = np.array([])

# #The sugars:
# sug_low = 68
# sug_high = 400

# for i in range(num_cycles):
#     if (i%2==0):
#         temp_sug = np.concatenate((temp_sug,sug_low*np.array_split(sug_dyn,num_cycles)[i]))
#     elif (i%2==1):
#         temp_sug = np.concatenate((temp_sug,sug_high*np.array_split(sug_dyn,num_cycles)[i]))
# sug_dyn = temp_sug

# fig,ax = plt.subplots(1,1)

# ax.plot(TimeRange/tau,sug_dyn,c='k')
# ax.tick_params(axis='both', which='major', labelsize=12)
# ax.tick_params(axis='both', which='minor', labelsize=10)
# # ax.set_xlabel(r'Time $t$',fontsize=18)
# # ax.set_ylabel('Sugar $s$',fontsize=18)
# ax.set_title('Sugar vs. time', fontsize=20)
# ax.set_box_aspect(1)
# ax.set_xticks((1,3))

In [6]:
# #Finding the g_opt as a function of time through sugar:
# g_opt_dyn = np.ones_like(sug_dyn)

# g_opt_low = g[np.argmax(fitness(g,sug_low,delta))]
# g_opt_high = g[np.argmax(fitness(g,sug_high,delta))]

# temp_g_opt = np.array([])
# for i in range(num_cycles):
#     if (i%2==0):
#         temp_g_opt = np.concatenate((temp_g_opt,g_opt_low*np.array_split(g_opt_dyn,num_cycles)[i]))
#     elif (i%2==1):
#         temp_g_opt = np.concatenate((temp_g_opt,g_opt_high*np.array_split(g_opt_dyn,num_cycles)[i]))
# g_opt_dyn = temp_g_opt

# # for sug_idx in range(len(sug_dyn)):
# #     g_opt_dyn[sug_idx] = g[np.argmax(fitness(g,sug_dyn[sug_idx],delta))]

# # G,S = np.meshgrid(g,sug_dyn)
# # fitness_2d = fitness(G,S,delta)
# # g_opt_dyn = g[np.argmax(fitness_2d,axis=1)]

# fig,ax = plt.subplots(1,1)
# ax.set_ylim(bottom=g_min,top=g_max)
# ax.plot(TimeRange/tau,g_opt_dyn,'k')
# ax.tick_params(axis='both', which='major', labelsize=12)
# ax.tick_params(axis='both', which='minor', labelsize=10)
# ax.set_xlabel(r'Time $t$',fontsize=18)
# ax.set_ylabel(r'$\hat{g}_s$',fontsize=18)
# ax.set_title('Optimal expression vs. time', fontsize=20)
# ax.set_xticks((0,2,4))
# ax.set_box_aspect(1)

In [7]:
# # dyn_p_unreg, dyn_p_reg, dyn_fit_unreg, dyn_norm_fit_unreg, dyn_fit_reg, dyn_norm_fit_reg = selection_dyn_const_noise(p0_unreg,p0_reg,sug_dyn,K,alpha,delta,T,dt)
# dyn_fit_unreg, dyn_fit_reg = selection_dyn_const_noise(p0_unreg,p0_reg,sug_dyn,K,alpha,delta,T,dt)[2:]
# overall_fitness_unreg = np.trapz(dyn_fit_unreg,dx=dt)
# overall_fitness_reg = np.trapz(dyn_fit_reg,dx=dt)

In [8]:
# #Now varying K, delta, alpha:
# K_arr = np.array([0.02,0.05,0.1,0.2,0.5])
# delta_arr = np.array([0.1,0.5,2.5,12.5])
# alpha_arr = np.array([0.5,4.5,12.5])

# long_term_fit_unreg = np.zeros((len(K_arr),len(delta_arr),len(alpha_arr)))
# long_term_fit_reg = np.zeros((len(K_arr),len(delta_arr),len(alpha_arr)))

# for K_idx in range(len(K_arr)):
#     for delta_idx in range(len(delta_arr)):
#         for alpha_idx in range(len(alpha_arr)):

#             #Now we initialize the parameters:
#             K = K_arr[K_idx]
#             delta = delta_arr[delta_idx]
#             alpha = alpha_arr[alpha_idx]
#             tau = 10

#             #Constant environment, T fixed:
#             dt = np.minimum(0.00005/K,0.005/delta)
#             # dt = 0.0001/K
#             # T = 10
#             # #If tau is too large, we don't change:
#             num_cycles = 8
#             T = num_cycles*tau
#             TimeRange = np.arange(0,T,dt)

#             # The noise variance D:
#             D = alpha*K*g_peak

# #             #Checks:
# #             print(f"K = {K}; 0.5*dg/dt = {0.5*dg/dt}")
# #             print(f"D = {D}; 0.5*dg**2/dt = {0.5*dg**2/dt}")

#             #Stability check:
#             flag_stability=0
#             if (K>=int(0.5*dg/dt) or D>=int(0.5*dg**2/dt)):
#                 flag_stability=1
#                 print("Warning! FTCS unstable.")
                
#             print(f"K,delta,alpha: {K, delta, alpha}\n")

#             #The initial distributions:
#             p0_unreg = Gaussian(g,g_peak,np.maximum(np.sqrt(alpha*g_peak),1))
#             p0_unreg = p0_unreg/np.trapz(p0_unreg,dx=dg)
#             p0_reg = p0_unreg
#             p0_reg = p0_reg/np.trapz(p0_reg,dx=dg)
            
#             #The dynamic sugar environment:
#             flag = False
#             sug_dyn = np.ones_like(TimeRange)
#             temp_sug = np.array([])
#             #The sugars:
#             sug_low = 68
#             sug_high = 400
#             for i in range(num_cycles):
#                 if (i%2==0):
#                     temp_sug = np.concatenate((temp_sug,sug_low*np.array_split(sug_dyn,num_cycles)[i]))
#                 elif (i%2==1):
#                     temp_sug = np.concatenate((temp_sug,sug_high*np.array_split(sug_dyn,num_cycles)[i]))
#             sug_dyn = temp_sug
            
#             #The optimal g:
#             g_opt_dyn = np.ones_like(sug_dyn)
#             g_opt_low = g[np.argmax(fitness(g,sug_low,delta))]
#             g_opt_high = g[np.argmax(fitness(g,sug_high,delta))]
#             temp_g_opt = np.array([])
#             for i in range(num_cycles):
#                 if (i%2==0):
#                     temp_g_opt = np.concatenate((temp_g_opt,g_opt_low*np.array_split(g_opt_dyn,num_cycles)[i]))
#                 elif (i%2==1):
#                     temp_g_opt = np.concatenate((temp_g_opt,g_opt_high*np.array_split(g_opt_dyn,num_cycles)[i]))
#             g_opt_dyn = temp_g_opt
            
#             #Now solving the equation:
#             long_term_fit_unreg[K_idx,delta_idx,alpha_idx],long_term_fit_reg[K_idx,delta_idx,alpha_idx] = selection_dyn_scalar_output(p0_unreg,p0_reg,sug_dyn,K,alpha,delta,T,dt)

## The parameter scans of excess fitness by regulation vs $K \tau$, $\delta \tau$, and $\alpha$.

In [9]:
# # #To save:
# # #np.savetxt("dynamic_env_long_term_fit_unreg_12012023.txt",long_term_fit_unreg.flatten())
# # #np.savetxt("dynamic_env_long_term_fit_reg_12012023.txt",long_term_fit_reg.flatten())

# #To load:
# test1 = np.loadtxt("dynamic_env_long_term_fit_unreg_12012023.txt").reshape(5,4,3)
# test2 = np.loadtxt("dynamic_env_long_term_fit_reg_12012023.txt").reshape(5,4,3)

In [10]:
# pic_vmin = np.min(long_term_fit_reg/long_term_fit_unreg - 1)*0.98
# pic_vmax = np.max(long_term_fit_reg/long_term_fit_unreg - 1)*1.02

In [11]:
# #Let's visualize the output:
# # plt.imshow(fit_norm_unreg.T,cmap=no_reg_cmp,origin="lower")
# alpha_idx = 0
# fig,ax = plt.subplots(1,1,constrained_layout='true')
# pic=ax.imshow(long_term_fit_reg[:,:,alpha_idx]/long_term_fit_unreg[:,:,alpha_idx] - 1,cmap='gray',origin='lower',vmin=pic_vmin,vmax=pic_vmax)
# cbar = plt.colorbar(pic)
# ax.set_xticks(ticks=range(4),labels=np.around(delta_arr*tau,decimals=2))
# ax.set_yticks(ticks=range(5),labels=np.around(K_arr*tau,decimals=2))
# ax.tick_params(axis='both', which='major', labelsize=12)
# ax.tick_params(axis='both', which='minor', labelsize=10)
# ax.set_ylabel(r'$K \tau$',fontsize=18)
# ax.set_xlabel(r'$\delta \tau$',fontsize=18)
# ax.set_title(fr'$\alpha = {np.around(np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max,2)}$',fontsize=20)
# # fig.savefig(f"excess_fitness_by_reg_alpha={np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max}.pdf",format="pdf",dpi=400,bbox_inches="tight")

In [12]:
# alpha_idx = 1
# fig,ax = plt.subplots(1,1,constrained_layout='true')
# pic=ax.imshow(long_term_fit_reg[:,:,alpha_idx]/long_term_fit_unreg[:,:,alpha_idx] - 1,cmap='gray',origin='lower',vmin=pic_vmin,vmax=pic_vmax)
# cbar = plt.colorbar(pic)
# ax.set_xticks(ticks=range(4),labels=np.around(delta_arr*tau,decimals=2))
# ax.set_yticks(ticks=range(5),labels=np.around(K_arr*tau,decimals=2))
# ax.tick_params(axis='both', which='major', labelsize=12)
# ax.tick_params(axis='both', which='minor', labelsize=10)
# # ax.set_ylabel(r'$K \tau$',fontsize=18)
# ax.set_xlabel(r'$\delta \tau$',fontsize=18)
# ax.set_title(fr'$\alpha = {np.around(np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max,2)}$',fontsize=20)
# # fig.savefig(f"excess_fitness_by_reg_alpha={np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max}.pdf",format="pdf",dpi=400,bbox_inches="tight")

In [13]:
# alpha_idx = 2
# fig,ax = plt.subplots(1,1,constrained_layout='true')
# pic=ax.imshow(long_term_fit_reg[:,:,alpha_idx]/long_term_fit_unreg[:,:,alpha_idx] - 1,cmap='gray',origin='lower',vmin=pic_vmin,vmax=pic_vmax)
# cbar = plt.colorbar(pic)
# ax.set_xticks(ticks=range(4),labels=np.around(delta_arr*tau,decimals=2))
# ax.set_yticks(ticks=range(5),labels=np.around(K_arr*tau,decimals=2))
# ax.tick_params(axis='both', which='major', labelsize=12)
# ax.tick_params(axis='both', which='minor', labelsize=10)
# # ax.set_ylabel(r'$K \tau$',fontsize=18)
# ax.set_xlabel(r'$\delta \tau$',fontsize=18)
# ax.set_title(fr'$\alpha = {np.around(np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max,2)}$',fontsize=20)
# # fig.savefig(f"excess_fitness_by_reg_alpha={np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max}.pdf",format="pdf",dpi=400,bbox_inches="tight")

## Excess fitness for fixed $\delta / K$ for various $\alpha$.

In [5]:
K_arr = np.logspace(start=np.log10(0.02),stop=np.log10(0.5),num=7)
delta_by_K_arr = np.array([1,5,25])
alpha_arr = np.array([0.5,4.5,12.5])

# long_term_fit_unreg = np.zeros((len(K_arr),len(delta_by_K_arr),len(alpha_arr)))
# long_term_fit_reg = np.zeros((len(K_arr),len(delta_by_K_arr),len(alpha_arr)))

# for K_idx in range(len(K_arr)):
#     for delta_by_K_idx in range(len(delta_by_K_arr)):
#         for alpha_idx in range(len(alpha_arr)):

#             #Now we initialize the parameters:
#             K = K_arr[K_idx]
#             delta = delta_by_K_arr[delta_by_K_idx]*K
#             alpha = alpha_arr[alpha_idx]
#             tau = 10

#             #Constant environment, T fixed:
#             dt = np.minimum(0.00005/K,0.005/delta)
#             # dt = 0.0001/K
#             # T = 10
#             # #If tau is too large, we don't change:
#             num_cycles = 8
#             T = num_cycles*tau
#             TimeRange = np.arange(0,T,dt)

#             # The noise variance D:
#             D = alpha*K*g_peak

# #             #Checks:
# #             print(f"K = {K}; 0.5*dg/dt = {0.5*dg/dt}")
# #             print(f"D = {D}; 0.5*dg**2/dt = {0.5*dg**2/dt}")

#             #Stability check:
#             flag_stability=0
#             if (K>=int(0.5*dg/dt) or D>=int(0.5*dg**2/dt)):
#                 flag_stability=1
#                 print("Warning! FTCS unstable.")
                
#             print(f"K,delta,alpha: {K, delta, alpha}")

#             #The initial distributions:
#             p0_unreg = Gaussian(g,g_peak,np.maximum(np.sqrt(alpha*g_peak),1))
#             p0_unreg = p0_unreg/np.trapz(p0_unreg,dx=dg)
#             p0_reg = p0_unreg
#             p0_reg = p0_reg/np.trapz(p0_reg,dx=dg)
            
#             #The dynamic sugar environment:
#             flag = False
#             sug_dyn = np.ones_like(TimeRange)
#             temp_sug = np.array([])
#             #The sugars:
#             sug_low = 68
#             sug_high = 400
#             for i in range(num_cycles):
#                 if (i%2==0):
#                     temp_sug = np.concatenate((temp_sug,sug_low*np.array_split(sug_dyn,num_cycles)[i]))
#                 elif (i%2==1):
#                     temp_sug = np.concatenate((temp_sug,sug_high*np.array_split(sug_dyn,num_cycles)[i]))
#             sug_dyn = temp_sug
            
#             #The optimal g:
#             g_opt_dyn = np.ones_like(sug_dyn)
#             g_opt_low = g[np.argmax(fitness(g,sug_low,delta))]
#             g_opt_high = g[np.argmax(fitness(g,sug_high,delta))]
#             temp_g_opt = np.array([])
#             for i in range(num_cycles):
#                 if (i%2==0):
#                     temp_g_opt = np.concatenate((temp_g_opt,g_opt_low*np.array_split(g_opt_dyn,num_cycles)[i]))
#                 elif (i%2==1):
#                     temp_g_opt = np.concatenate((temp_g_opt,g_opt_high*np.array_split(g_opt_dyn,num_cycles)[i]))
#             g_opt_dyn = temp_g_opt
            
#             #Now solving the equation:
#             long_term_fit_unreg[K_idx,delta_by_K_idx,alpha_idx],long_term_fit_reg[K_idx,delta_by_K_idx,alpha_idx] = selection_dyn_scalar_output(p0_unreg,p0_reg,sug_dyn,K,alpha,delta,T,dt)
            
#             print(f"long term fitness, unreg and reg: {long_term_fit_unreg[K_idx,delta_by_K_idx,alpha_idx],long_term_fit_reg[K_idx,delta_by_K_idx,alpha_idx]}\n")

In [4]:
# # # # #To save:
# # # # np.savetxt("long_term_fit_unreg_fixed_delta-K_ratio_13012023.txt",long_term_fit_unreg.flatten())
# # # # np.savetxt("long_term_fit_reg_fixed_delta-K_ratio_13012023.txt",long_term_fit_reg.flatten())

# #To load:
# result1 = np.loadtxt("long_term_fit_unreg_fixed_delta-K_ratio_13012023.txt").reshape(7,3,3)
# result2 = np.loadtxt("long_term_fit_reg_fixed_delta-K_ratio_13012023.txt").reshape(7,3,3)

In [146]:
# tau = 10
# ylim_max = np.max(result2/result1 - 1)*1.1
# ylim_min = np.min(result2/result1 - 1)*0.4

# fit_params = np.zeros((len(delta_by_K_arr),len(alpha_arr),3))

# for delta_by_K_idx in range(len(delta_by_K_arr)):
#     for alpha_idx in range(len(alpha_arr)):
#         popt,pcov = sciopt.curve_fit(logistic_sigmoid,K_arr,(result2/result1 - 1)[:,delta_by_K_idx,alpha_idx],p0=([1,1,1]),maxfev = 10000)
#         fit_params[delta_by_K_idx,alpha_idx] = popt
        
# K_axis = np.logspace(start=np.log10(0.02),stop=np.log10(0.5),num=51)

# colors_grey = ['#b3b3b3','#666666','#000000']

In [147]:
# delta_by_K_idx = 0

# fig,ax = plt.subplots(1,1,constrained_layout='true')

# for alpha_idx in range(len(alpha_arr)):
#     ax.scatter(K_arr,(result2/result1 - 1)[:,delta_by_K_idx,alpha_idx],c=colors_grey[alpha_idx])
#     ax.plot(K_axis,logistic_sigmoid(K_axis,*fit_params[delta_by_K_idx,alpha_idx]),c=colors_grey[alpha_idx],ls='--',\
#             label=fr'max = {np.around(fit_params[delta_by_K_idx,alpha_idx,0],2)}')

# ax.set_ylim([ylim_min,ylim_max])
# ax.set_xscale("log")
# ax.set_xticks(K_arr)
# ax.set_xticklabels(np.around(K_arr*tau,1))
# ax.minorticks_off()
# ax.set_box_aspect(1)
# ax.set_xlabel(r'$K \tau$',fontsize=18)
# ax.set_ylabel(r'excess fitness',fontsize=18)
# ax.set_title(fr'$\delta / K = {delta_by_K_arr[delta_by_K_idx]}$',fontsize=20)
# ax.legend(loc='best')
# fig.set_figheight(3.6)
# fig.savefig(f"ex_fit_del_by_K={delta_by_K_arr[delta_by_K_idx]}.pdf",format="pdf",dpi=400,bbox_inches="tight")

In [148]:
# delta_by_K_idx = 1

# fig,ax = plt.subplots(1,1,constrained_layout='true')

# for alpha_idx in range(len(alpha_arr)):
#     ax.scatter(K_arr,(result2/result1 - 1)[:,delta_by_K_idx,alpha_idx],c=colors_grey[alpha_idx])
#     ax.plot(K_axis,logistic_sigmoid(K_axis,*fit_params[delta_by_K_idx,alpha_idx]),c=colors_grey[alpha_idx],ls='--',\
#             label=fr'max = {np.around(fit_params[delta_by_K_idx,alpha_idx,0],2)}')

# ax.set_ylim([ylim_min,ylim_max])
# ax.set_xscale("log")
# ax.set_xticks(K_arr)
# ax.set_xticklabels(np.around(K_arr*tau,1))
# ax.minorticks_off()
# ax.set_box_aspect(1)
# ax.set_xlabel(r'$K \tau$',fontsize=18)
# # ax.set_ylabel(r'excess fitness',fontsize=18)
# ax.set_title(fr'$\delta / K = {delta_by_K_arr[delta_by_K_idx]}$',fontsize=20)
# ax.legend(loc='best')
# fig.set_figheight(3.6)
# fig.savefig(f"ex_fit_del_by_K={delta_by_K_arr[delta_by_K_idx]}.pdf",format="pdf",dpi=400,bbox_inches="tight")

In [149]:
# delta_by_K_idx = 2

# fig,ax = plt.subplots(1,1,constrained_layout='true')

# for alpha_idx in range(len(alpha_arr)):
#     ax.scatter(K_arr,(result2/result1 - 1)[:,delta_by_K_idx,alpha_idx],c=colors_grey[alpha_idx])
#     ax.plot(K_axis,logistic_sigmoid(K_axis,*fit_params[delta_by_K_idx,alpha_idx]),c=colors_grey[alpha_idx],ls='--',\
#             label=fr'$\alpha = {np.sqrt(alpha_arr[alpha_idx]*g_peak)/g_max}$, max = {np.around(fit_params[delta_by_K_idx,alpha_idx,0],2)}')
    
# ax.set_ylim([ylim_min,ylim_max])
# ax.set_xscale("log")
# ax.set_xticks(K_arr)
# ax.set_xticklabels(np.around(K_arr*tau,1))
# ax.minorticks_off()
# ax.set_box_aspect(1)
# ax.set_xlabel(r'$K \tau$',fontsize=18)
# # ax.set_ylabel(r'excess fitness',fontsize=18)
# ax.set_title(fr'$\delta / K = {delta_by_K_arr[delta_by_K_idx]}$',fontsize=20)
# ax.legend(loc='best')
# fig.set_figheight(3.6)
# fig.savefig(f"ex_fit_del_by_K={delta_by_K_arr[delta_by_K_idx]}.pdf",format="pdf",dpi=400,bbox_inches="tight")